# Stochastic optimization model
## Imports

In [1]:
# Import packages
using MLDataUtils, CSV, DataFrames,Plots, Statistics, StatsBase
using Gurobi, JuMP
using Random

In [2]:
# Import data 
df = DataFrame(CSV.read("Game_data/2022_LoL_esports_match_data_from_OraclesElixir_20221016.csv", DataFrame, pool = true))
utility_B = DataFrame(CSV.read("Utility/U_B.csv", DataFrame, pool = true))
utility_R = DataFrame(CSV.read("Utility/U_R.csv", DataFrame, pool = true))
ban_vec = DataFrame(CSV.read("Utility/ban_vec.csv", DataFrame, pool = true))
ban_mat = DataFrame(CSV.read("Utility/ban_mat.csv", DataFrame, pool = true))
pick_mat = DataFrame(CSV.read("Utility/pick_mat.csv", DataFrame, pool = true));

In [3]:
n = size(utility_B)[1] # number of champions
m = size(utility_B)[2] -1 # number of positions
T = 1:20; # number of events

In [4]:
utility_B

Row,champion,bot,jng,mid,sup,top
,String15,Float64,Float64,Float64,Float64,Float64
1,Aatrox,0.0,0.0,0.0,0.0,1.0
2,Ahri,0.0,0.0,3.16228,0.0,0.0
3,Akali,0.0,0.0,1.0,0.0,1.41421
4,Alistar,0.0,0.0,0.0,1.0,0.0
5,Amumu,0.0,0.0,0.0,2.0,0.0
6,Aphelios,3.0,0.0,0.0,0.0,0.0
7,Azir,0.0,0.0,1.41421,0.0,0.0
8,Braum,0.0,0.0,0.0,1.0,0.0
9,Corki,0.0,0.0,1.0,0.0,0.0


Lets see how we can find the maximum utility of a given champion using the utility matrix.

In [5]:
# champion with highest utility 
maximum(utility_B[:,1])

"Zoe"

In [6]:
# find utility of "Zoe"
utility_B[utility_B[:,1] .== "Zoe",2:end]

Row,bot,jng,mid,sup,top
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,2.23607,0.0,0.0


In [7]:
# above as Array
maximum(Array(utility_B[utility_B[:,1] .== "Zoe",2:end]))

2.23606797749979

In [8]:
utility_B[:,2:end]

Row,bot,jng,mid,sup,top
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,3.16228,0.0,0.0
3,0.0,0.0,1.0,0.0,1.41421
4,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,2.0,0.0
6,3.0,0.0,0.0,0.0,0.0
7,0.0,0.0,1.41421,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0
9,0.0,0.0,1.0,0.0,0.0


## Implementation of the model

In [135]:
function stochastic(df, ban_mat, pick_mat)

    # Define model
    model = Model(Gurobi.Optimizer)

    # Define the sets 
    n = size(utility_B)[1] # number of champions
    m = size(utility_B)[2] -1 # number of positions
    T = 1:20 # number of events
    tbb = [1,3,5,14,16] # blue team ban events
    trb = [2,4,6,13,15] # red team ban events
    tbp = [7,10,11,18,19] # blue team pick events
    trp = [8,9,12,17,20] # red team pick events

    # Define the parameters
    epsilon = 0.001 # small number

    # Define the independant variables
    UB = Array(utility_B[:,2:end]) # blue team utility
    UR = Array(utility_R[:,2:end]) # red team utility
    
    global BU_T = 0 # total utility of blue team
    global RU_T = 0 # total utility of red team

    # Define the dependant variables
    @variable(model, pool = zeros(n,m)) # matrix of champions in the pool
    @variable(model, bans = zeros(n,m)) # matrix of champions banned to understand order of choices
    @variable(model, picks = zeros(n,m)) # matrix of champions picked to understand order of choices

    @variable(model, BB[1:n,1:m,1:T], Bin) # blue ban of champion i at position j at event t
    # @variable(model, RB[1:n,1:m,1:T], Bin) # red ban of champion i at position j at event t --> obtained probabilistically
    @variable(model, BP[1:n,1:m,1:T], Bin) # blue pick of champion i at position j at event t
    # @variable(model, RP[1:n,1:m,1:T], Bin) # red pick of champion i at position j at event t --> obtained probabilistically

    @variable(model, z[1:T], Bin) # binary variable to determine if event t is a ban or a pick

    # Define the constraints
    # 1. Each champion can only be picked/banned at most once
    @constraint(model, [i=1:n], sum(sum((BB[i,j,t] + RB[i,j,t])*z[t] + (BP[i,jt] + RP[i,j,t])*(1-z[t]) for j = 1:m) for t=1:T) <= 1)

    # 2. Each team must ban at most 5 champions
    @constraint(model, [j=1:m], sum(sum(BB[i,j,t] for i = 1:n) for t in tbb) <= 5)
    @constraint(model, [j=1:m], sum(sum(RB[i,j,t] for i = 1:n) for t in trb) <= 5)

    # 3. Each team must pick 5 champions
    @constraint(model, sum(sum(sum(BP[i,j,t] for i = 1:n) for j = 1:m) for t in tbp) == 5)
    @constraint(model, sum(sum(sum(RP[i,j,t] for i = 1:n) for j = 1:m) for t in trp) == 5)

    # 4. Each team must pick 1 champion per position
    @constraint(model, [j=1:m], sum(sum(BP[i,j,t] for i = 1:n) for t in tbp)== 1)
    @constraint(model, [j=1:m], sum(sum(RP[i,j,t] for i = 1:n) for t in trp) == 1)

    # 5. The utility of a champion picked or banned must be greater than epsilon
    @constraint(model, [i=1:n, j=1:m], UB[i,j] >= epsilon)
    @constraint(model, [i=1:n, j=1:m], UR[i,j] >= epsilon)

    # 6. The red ban will be the champion with the highest utility for the blue team 
    @constraint(model, [t=trb], RB[i,j,:] = findmax(UB)[2] == [1,1,:])

    # 7. The red pick will be the champion with the highest utility for the red team at the position of the blue team's pick
    # @constraint(model, [t=trp], RP[i,j,:] = findmax(UR)[2]) 

    # 8. Constraint on bans/picks for the objective function
    @constraint(model, [t=tbb], z[t] == 1)
    @constraint(model, [t=trb], z[t] == 1)
    @constraint(model, [t=tbp], z[t] == 0)
    @constraint(model, [t=trp], z[t] == 0)

    for t in 1:T
        if t in tbb
            @constraint(model, [i=1:n, j=1:m], BB[i,j,t] <= bans[i,j])
            @constraint(model, bans[i,j] <= pool[i,j])
        elseif t in trb
            @constraint(model, [i=1:n, j=1:m], RB[i,j,t] <= bans[i,j])
            @constraint(model, bans[i,j] <= pool[i,j])
        elseif t in tbp
            @constraint(model, [i=1:n, j=1:m], BP[i,j,t] <= picks[i,j])
            @constraint(model, picks[i,j] <= pool[i,j])
        elseif t in trp
            @constraint(model, [i=1:n, j=1:m], RP[i,j,t] <= picks[i,j])
            @constraint(model, picks[i,j] <= pool[i,j])
        end
    end

    # for t in 1:6
    #     # The ban of red team will be the champion with the biggest utility in UB
    #     RB[:,1,t] = maximum(Array(UB[UB[:,1] .== maximum(UB[:,1]),2:end]))

    #     # Define specific objective for this series of events
        @objective(model, Max, sum(sum(UB[i,j] * BB[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UB[i,j] * BP[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UR[i,j] * RB[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UR[i,j] * RP[i,j,t] for i = 1:n) for j = 1:m))

    #     # Solve the model
    #     optimize!(model)

    #     # Get optimal value of UB
    #     BU_T += objective_value(model)

    #     # Update the pool
    #     pool = pool + value.(BB[:,:,t]) + value.(BP[:,:,t]) + value.(RB[:,:,t]) + value.(RP[:,:,t])

    #     # Update the bans
    #     bans = bans + value.(BB[:,:,t]) + value.(RB[:,:,t])

    #     # Update the picks  
    #     picks = picks + value.(BP[:,:,t]) + value.(RP[:,:,t])


end

LoadError: LoadError: At In[135]:58: `@constraint(model, [t = trb], RB[i, j, :] = (findmax(UB))[2] == [1, 1, :])`: No constraint expression was given.
in expression starting at In[135]:58

OTHERWISE DO TWO FUNCTIONS AND THEN A FOR LOOp WITH 20 STATEMENTS ??

In [130]:
findmax(UB)[2]

CartesianIndex(34, 4)

In [107]:
UB = Array(utility_B[:,2:end])

# above as Array
# maximum(Array(UB[UB[:,1] .== "Zoe",2:end]))
maximum(UB)

# index of maximum
maximum(UB) == UB


4.123105625617661

In [124]:
findmax(UB)

(4.123105625617661, CartesianIndex(34, 4))

In [119]:
# maximum of UB for each column 
maximum(UB, dims=1)

1×5 Matrix{Float64}:
 3.74166  3.74166  3.16228  4.12311  3.31662

In [108]:
UB

62×5 Matrix{Float64}:
 0.0      0.0      0.0      0.0      1.0
 0.0      0.0      3.16228  0.0      0.0
 0.0      0.0      1.0      0.0      1.41421
 0.0      0.0      0.0      1.0      0.0
 0.0      0.0      0.0      2.0      0.0
 3.0      0.0      0.0      0.0      0.0
 0.0      0.0      1.41421  0.0      0.0
 0.0      0.0      0.0      1.0      0.0
 0.0      0.0      1.0      0.0      0.0
 0.0      1.73205  0.0      0.0      0.0
 1.41421  0.0      0.0      0.0      0.0
 1.73205  0.0      0.0      0.0      0.0
 0.0      0.0      0.0      0.0      2.64575
 ⋮                                   
 0.0      0.0      1.41421  0.0      0.0
 0.0      0.0      2.82843  0.0      0.0
 0.0      1.0      0.0      0.0      0.0
 0.0      3.16228  0.0      0.0      0.0
 0.0      0.0      2.44949  0.0      0.0
 0.0      1.73205  0.0      0.0      0.0
 0.0      2.0      0.0      0.0      0.0
 1.73205  0.0      0.0      0.0      0.0
 0.0      3.74166  0.0      0.0      0.0
 0.0      0.0      0.0      1.

In [ ]:
# Champion with highest utility 
maximum(utility_B[:,1])

# find utility of "Zoe"
utility_B[utility_B[:,1] .== "Zoe",2:end]

# above as Array
maximum(Array(utility_B[utility_B[:,1] .== "Zoe",2:end]))

In [95]:
maximum(Array(utility_B[utility_B[:,1] .== maximum(utility_B[:,1]),2:end]))

2.23606797749979

## Banning phase

Introduce a binary variable for the different optimisations !!!

In [ ]:
function stochastic(df, ban_mat, pick_mat)

    # Define model
    model = Model(Gurobi.Optimizer)

    # Define the sets 
    n = size(utility_B)[1] # number of champions
    m = size(utility_B)[2] -1 # number of positions
    T = 1:20 # number of events
    tbb = [1,3,5,14,16] # blue team ban events
    trb = [2,4,6,13,15] # red team ban events
    tbp = [7,10,11,18,19] # blue team pick events
    trp = [8,9,12,17,20] # red team pick events

    # Define the parameters
    epsilon = 0.001 # small number

    # Define the independant variables
    UB = Array(utility_B[:,2:end]) # blue team utility
    UR = Array(utility_R[:,2:end]) # red team utility
    
    global BU_T = 0 # total utility of blue team
    global RU_T = 0 # total utility of red team

    global pool = zeros(n,m) # binary matrix of champions in the pool
    global bans = zeros(n,m) # binary matrix of champions banned to understand order of choices
    global picks = zeros(n,m) # binary matrix of champions picked to understand order of choices

    global RB ????
    @variable(model, BB[1:n,1:m,t:T], Bin) # blue ban of champion i at position j at event t
    # @variable(model, RB[1:n,1:m,t:T], Bin) # red ban of champion i at position j at event t --> obtained probabilistically
    @variable(model, BP[1:n,1:m,t:T], Bin) # blue pick of champion i at position j at event t
    # @variable(model, RP[1:n,1:m,t:T], Bin) # red pick of champion i at position j at event t --> obtained probabilistically

    # Define the constraints
    # 1. Each champion can only be picked/banned at most once
    @constraint(model, [i=1:n], sum(sum(BB[i,j,t] + RB[i,j,t] + BP[i,jt] + RP[i,j,t] for j = 1:m ) for t=1:T) <= 1)

    # 2. Each team must ban at most 5 champions
    @constraint(model, [j=1:m], sum(sum(BB[i,j,t] for i = 1:n) for t in tbb) <= 5)
    @constraint(model, [j=1:m], sum(sum(RB[i,j,t] for i = 1:n) for t in trb) <= 5)

    # 3. Each team must pick 5 champions
    @constraint(model, sum(sum(sum(BP[i,j,t] for i = 1:n) for j = 1:m) for t in tbp) == 5)
    @constraint(model, sum(sum(sum(RP[i,j,t] for i = 1:n) for j = 1:m) for t in trp) == 5)

    # 4. Each team must pick 1 champion per position
    @constraint(model, [j=1:m], sum(sum(BP[i,j,t] for i = 1:n) for t in tbp)== 1)
    @constraint(model, [j=1:m], sum(sum(RP[i,j,t] for i = 1:n) for t in trp) == 1)

    # 5. The utility of a champion picked or banned must be greater than epsilon
    @constraint(model, [i=1:n, j=1:m], UB[i,j] >= epsilon)
    @constraint(model, [i=1:n, j=1:m], UR[i,j] >= epsilon)

    # 6. The red ban will be the champion with the highest utility for the blue team 
    @constraint(model, [t=trb], RB[i,j,:] = findmax(UB)[2])

    # 7. The red pick will be the champion with the highest utility for the red team at the position of the blue team's pick
    @constraint(model, [t=trp], RP[i,j,:] = findmax(UR)[2]) 

    # for t in 1:6
    #     # The ban of red team will be the champion with the biggest utility in UB
    #     RB[:,1,t] = maximum(Array(UB[UB[:,1] .== maximum(UB[:,1]),2:end]))

    #     # Define specific objective for this series of events
    #     @objective(model, Max, sum(sum(UB[i,j] * BB[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UB[i,j] * BP[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UR[i,j] * RB[i,j,t] for i = 1:n) for j = 1:m) + sum(sum(UR[i,j] * RP[i,j,t] for i = 1:n) for j = 1:m))

    #     # Solve the model
    #     optimize!(model)

    #     # Get optimal value of UB
    #     BU_T += objective_value(model)

    #     # Update the pool
    #     pool = pool + value.(BB[:,:,t]) + value.(BP[:,:,t]) + value.(RB[:,:,t]) + value.(RP[:,:,t])

    #     # Update the bans
    #     bans = bans + value.(BB[:,:,t]) + value.(RB[:,:,t])

    #     # Update the picks  
    #     picks = picks + value.(BP[:,:,t]) + value.(RP[:,:,t])


end